In [3]:
import os

print(os.listdir(extract_path))  # Should show ['gender_dataset']
print(os.listdir(f"{extract_path}/Training"))  # Should show ['male', 'female']


['Training', 'Validation']
['female', 'male']


In [4]:
import os
import torch
import torchvision
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from torch import nn, optim
from tqdm import tqdm
import copy


In [5]:
# ImageNet mean & std for normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'Training': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.RandomRotation(15),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'Validation': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}


In [6]:
data_dir = '/content/gender_dataset'
batch_size = 64

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['Training', 'Validation']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=2)
    for x in ['Training', 'Validation']
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['Training', 'Validation']}
class_names = image_datasets['Training'].classes

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [7]:
model = models.resnet18(pretrained=True)

# Freeze earlier layers (optional for small datasets)
for param in model.parameters():
    param.requires_grad = False

# Replace the classifier
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)  # 2 classes: male, female

model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 162MB/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [9]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=3):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    epochs_no_improve = 0
    min_val_loss = float('inf')

    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 30)

        for phase in ['Training', 'Validation']:
            model.train() if phase == 'Training' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            loop = tqdm(dataloaders[phase], desc=f"{phase}", leave=False)
            for inputs, labels in loop:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'Training'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'Training':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Early stopping & checkpoint
            if phase == 'Validation':
                scheduler.step(epoch_loss)
                if epoch_loss < min_val_loss:
                    min_val_loss = epoch_loss
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    epochs_no_improve = 0
                    torch.save(model.state_dict(), 'best_model.pth')
                else:
                    epochs_no_improve += 1
                    if epochs_no_improve >= patience:
                        print("Early stopping triggered.")
                        model.load_state_dict(best_model_wts)
                        return model

    model.load_state_dict(best_model_wts)
    return model


In [10]:
model = train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=3)



Epoch 1/30
------------------------------


Training Loss: 0.6182 Acc: 0.6571


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:243: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Validation Loss: 0.5228 Acc: 0.7244

Epoch 2/30
------------------------------


Training Loss: 0.5439 Acc: 0.7295


...

Validation Loss: 0.3515 Acc: 0.8508

Epoch 22/30
------------------------------


Training Loss: 0.4651 Acc: 0.7777


Validation Loss: 0.3636 Acc: 0.8427

Epoch 23/30
------------------------------


Training Loss: 0.4649 Acc: 0.7763


Validation Loss: 0.3532 Acc: 0.8487
Early stopping triggered.


In [15]:
torch.save(model.state_dict(), "/content/resnet18_gender_classification.pth")
